In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import time
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_FINE_TUNE = 10

BASE_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_trashnet_base"
PRUNED_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40"

DATASET_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"

In [3]:
# DATA GENERATOR (RESCALE ONLY)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "test"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 2001 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [4]:
# FUNGSI CEK UKURAN MODEL
def get_model_size_mb(model_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(model_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)

In [5]:
base_model = tf.keras.models.load_model(
    BASE_MODEL_PATH,
    compile=False
)

base_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

base_model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                61504     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 3058246 (11.67 MB)
Trainable params: 61894 (241.77 KB)
Non-trainable params: 2996352 (11.43 MB)
_________________________________________________________________


In [6]:
# CEK UKURAN BASE MODEL
base_model_size = get_model_size_mb(BASE_MODEL_PATH)
print(f"Ukuran Base Model      : {base_model_size:.2f} MB")

Ukuran Base Model      : 16.03 MB


In [7]:
def apply_pruning(model, initial_sparsity, final_sparsity, end_step):

    pruning_params = {
        "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=initial_sparsity,
            final_sparsity=final_sparsity,
            begin_step=0,
            end_step=end_step
        )
    }

    def prune_layer(layer):
        if isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            return layer
        if isinstance(layer, tf.keras.layers.Conv2D):
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        return layer

    return tf.keras.models.clone_model(
        model,
        clone_function=prune_layer
    )

In [8]:
steps_per_epoch = np.ceil(train_data.samples / BATCH_SIZE)
end_step = int(steps_per_epoch * EPOCHS_FINE_TUNE)

In [9]:
pruned_model = apply_pruning(base_model, 0.0, 0.20, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    EarlyStopping(patience=2, restore_best_weights=True)
]

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10


63/63 [==============================] - 51s 300ms/step - loss: 0.1667 - accuracy: 0.9645 - val_loss: 0.4290 - val_accuracy: 0.8462
Epoch 2/10
63/63 [==============================] - 11s 181ms/step - loss: 0.1505 - accuracy: 0.9730 - val_loss: 0.4223 - val_accuracy: 0.8515
Epoch 3/10
63/63 [==============================] - 11s 182ms/step - loss: 0.1401 - accuracy: 0.9760 - val_loss: 0.4147 - val_accuracy: 0.8541
Epoch 4/10
63/63 [==============================] - 12s 183ms/step - loss: 0.1285 - accuracy: 0.9800 - val_loss: 0.4106 - val_accuracy: 0.8541
Epoch 5/10
63/63 [==============================] - 12s 184ms/step - loss: 0.1192 - accuracy: 0.9815 - val_loss: 0.4067 - val_accuracy: 0.8541
Epoch 6/10
63/63 [==============================] - 11s 181ms/step - loss: 0.1113 - accuracy: 0.9855 - val_loss: 0.4054 - val_accuracy: 0.8568
Epoch 7/10
63/63 [==============================] - 11s 181ms/step - loss: 0.1036 - accuracy: 0.9860 - val_loss: 0.3990 - val_accuracy: 0.85

In [10]:
pruned_model = apply_pruning(pruned_model, 0.20, 0.30, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10
63/63 [==============================] - 34s 293ms/step - loss: 0.0984 - accuracy: 0.9875 - val_loss: 0.4040 - val_accuracy: 0.8568
Epoch 2/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0894 - accuracy: 0.9900 - val_loss: 0.4000 - val_accuracy: 0.8594
Epoch 3/10
63/63 [==============================] - 11s 181ms/step - loss: 0.0824 - accuracy: 0.9915 - val_loss: 0.3944 - val_accuracy: 0.8621
Epoch 4/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0763 - accuracy: 0.9950 - val_loss: 0.3919 - val_accuracy: 0.8594
Epoch 5/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0711 - accuracy: 0.9965 - val_loss: 0.4004 - val_accuracy: 0.8568
Epoch 6/10
63/63 [==============================] - 12s 183ms/step - loss: 0.0660 - accuracy: 0.9970 - val_loss: 0.3982 - val_accuracy: 0.8621


In [11]:
pruned_model = apply_pruning(pruned_model, 0.30, 0.40, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10
63/63 [==============================] - 34s 303ms/step - loss: 0.0727 - accuracy: 0.9960 - val_loss: 0.3963 - val_accuracy: 0.8647
Epoch 2/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0657 - accuracy: 0.9955 - val_loss: 0.3951 - val_accuracy: 0.8594
Epoch 3/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0602 - accuracy: 0.9970 - val_loss: 0.4103 - val_accuracy: 0.8515
Epoch 4/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0559 - accuracy: 0.9975 - val_loss: 0.3950 - val_accuracy: 0.8647
Epoch 5/10
63/63 [==============================] - 11s 180ms/step - loss: 0.0517 - accuracy: 0.9980 - val_loss: 0.4024 - val_accuracy: 0.8674
Epoch 6/10
63/63 [==============================] - 11s 181ms/step - loss: 0.0478 - accuracy: 0.9980 - val_loss: 0.4093 - val_accuracy: 0.8594


In [12]:
final_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

In [13]:
final_pruned_model.save(
    PRUNED_MODEL_PATH,
    save_format="tf"
)

print("Model pruning berhasil disimpan")

INFO:tensorflow:Assets written to: D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40\assets


INFO:tensorflow:Assets written to: D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40\assets


Model pruning berhasil disimpan


In [14]:
# CEK UKURAN MODEL SETELAH PRUNING
pruned_model_size = get_model_size_mb(PRUNED_MODEL_PATH)
print(f"Ukuran Pruned Model    : {pruned_model_size:.2f} MB")

compression_ratio = base_model_size / pruned_model_size
size_reduction = (1 - (pruned_model_size / base_model_size)) * 100

print(f"Compression Ratio     : {compression_ratio:.2f}x")
print(f"Size Reduction        : {size_reduction:.2f}%")

Ukuran Pruned Model    : 15.55 MB
Compression Ratio     : 1.03x
Size Reduction        : 2.94%


In [15]:
test_data.reset()

predictions = final_pruned_model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = test_data.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

12/12 [==============================] - 9s 153ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88        61
           1       0.83      0.92      0.88        76
           2       0.86      0.87      0.86        62
           3       0.88      0.89      0.88        90
           4       0.83      0.82      0.83        72
           5       0.82      0.64      0.72        22

    accuracy                           0.86       383
   macro avg       0.86      0.83      0.84       383
weighted avg       0.86      0.86      0.86       383


Confusion Matrix:
[[52  0  0  8  0  1]
 [ 0 70  3  0  3  0]
 [ 0  3 54  3  2  0]
 [ 5  0  1 80  3  1]
 [ 0  8  4  0 59  1]
 [ 0  3  1  0  4 14]]


In [16]:
import time
import numpy as np

def benchmark_inference(model, data_gen, warmup=3, runs=10):
    """
    model      : stripped pruned model
    data_gen   : test_data (shuffle=False)
    warmup     : jumlah warmup run
    runs       : jumlah benchmark run
    """

    # Ambil 1 batch data
    x_batch, _ = next(data_gen)

    # Warm-up (penting untuk TensorFlow)
    for _ in range(warmup):
        _ = model.predict(x_batch, verbose=0)

    times = []

    for _ in range(runs):
        start = time.time()
        _ = model.predict(x_batch, verbose=0)
        end = time.time()
        times.append(end - start)

    avg_time = np.mean(times)
    per_image_time = avg_time / x_batch.shape[0]

    print("Inference Benchmark")
    print(f"Batch size         : {x_batch.shape[0]}")
    print(f"Avg batch time     : {avg_time:.4f} seconds")
    print(f"Avg per image time : {per_image_time:.6f} seconds")

    return avg_time, per_image_time

In [17]:
benchmark_inference(final_pruned_model, test_data)

Inference Benchmark
Batch size         : 32
Avg batch time     : 0.1957 seconds
Avg per image time : 0.006116 seconds


(0.19571435451507568, 0.006116073578596115)